# Creating a basic Cluster Expansion

In [1]:
import random
import numpy as np
from monty.serialization import loadfn, dumpfn
from pymatgen.core.structure import Structure
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion, RegressionData

No module named 'phonopy'
No module named 'phonopy'


In [7]:
# load the prim structure
#prim_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/v4cr4ti_prim_cell.json'
#prim_path = '/home/myless/Packages/structure_maker/vcrtiwzr_prim_entry.json'
prim_path = '/Users/myless/Packages/structure_maker/Entries/v1_6cr1_6ti1_6w1_6zr_prim_struct_dos.json'
lno_prim = loadfn(prim_path)
    
# load the fitting data
#entry_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/vcrti_fixed_entries.json'
#entry_path = '/home/myless/Packages/structure_maker/vcrtiwzr_cse.json'
entry_path = '/Users/myless/Packages/structure_maker/Entries/vcrtiwzr_entries.json'
lno_entries = loadfn(entry_path)

### 0) The prim structure
The prim structure defines the **configurational space** for the Cluster Expansion. 
The **configurational space** is defined by the site **compositional spaces** and the crystal symetries of the prim structure.
The occupancy of the sites determine site **compositional spaces**. Sites are **active** if they have compositional degrees of freedom.


Active sites have fractional compositions. Vacancies are allowed in sites where the composition does not sum to one.

0. Is active. The allowed species are: Li+ and vacancies.
1. Is active. The allowed species are: Ni3+ and Ni4+.
2. Is not active. Only O2- is allowed.
3. Is not active. Only O2- is allowed.

In [8]:
print(lno_prim)

Full Formula (Zr0.03124 Ti0.03124 V1.875 Cr0.03124 W0.03124)
Reduced Formula: Zr0.03124Ti0.03124V1.875Cr0.03124W0.03124
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016  0    0    0
  1  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016  0.5  0.5  0.5


### 1) The cluster subspace
The `ClusterSubspace` represents all the orbits (groups of equivalent clusters) that will be considered when fitting the cluster expansion. Its main purpose is to compute the **correlations functions** for each included orbit given a structure in the compositional space defined by the prim.

In order to do be able to compute the correlation functions, the given structure must match the prim structure in a "crystallographic" sense but allowing for compositional degrees of freedom in the "active" sites.

A cluster subspace most easily created by providing:
1. The prim structure representing the configurational space.
2. A set of diameter cutoffs for each size of orbit we want to consider.
3. A type of site basis function to use.

There are more options allowed by the code to fine grain and tune. See other notebooks for advanced use cases.

In [12]:
subspace = ClusterSubspace.from_cutoffs(
    lno_prim,
    cutoffs={2: 5, 3: 4.1, 4: 3.01, 5: 2}, # will include orbits of 2 and 3 sites.
    basis='sinusoid', # sets the site basis type, default is indicator
    supercell_size='num_sites'
)

# supercell_size specifies the method to determine the supercell size
# when trying to match a structure.
# (See pymatgen.structure_matcher.StructureMatcher for more info)

print(subspace) # single site and empty orbits are always included.

Basis/Orthogonal/Orthonormal : sinusoid/True/False
       Unit Cell Composition : Cr0.03124 Ti0.03124 W0.03124 Zr0.03124 V1.875
            Number of Orbits : 8
No. of Correlation Functions : 140
             Cluster Cutoffs : 2: 4.99, 3: 3.01, 4: 3.01
              External Terms : []
Orbit Summary
 ------------------------------------------------------------------------
 |  ID     Degree    Cluster Diameter    Multiplicity    No. Functions  |
 |   0       0             NA                 0                1        |
 |   1       1            0.0000              2                4        |
 |   2       2            2.6067              8               10        |
 |   3       2            3.0100              6               10        |
 |   4       2            4.2568              12              10        |
 |   5       2            4.9915              24              10        |
 |   6       3            3.0100              24              40        |
 |   7       4            3.0100 

#### 1.1) Computing a correlation vector.
A correlation vector for a specific structure (represents the feature vector) used to train and predict target values.

In [13]:
structure = lno_entries[1].structure
corr = subspace.corr_from_structure(structure)

print(f'The correlation vector for a structure with'
      f' composition {structure.composition} is: '
      f'\n{corr}')

The correlation vector for a structure with composition Zr3 Ti4 V47 Cr5 W5 is: 
[ 1.00000000e+00  5.66995793e-01 -3.70873814e-01 -1.84183293e-01
  6.33314983e-01  3.08007956e-01 -2.27934447e-01 -1.08398437e-01
  3.62085068e-01  1.51152532e-01  7.91306018e-02 -2.33651634e-01
  4.16014193e-02 -1.25468957e-01  3.95722468e-01  3.47182571e-01
 -2.07320919e-01 -1.00260417e-01  3.62334753e-01  1.26485512e-01
  6.01512809e-02 -2.24189107e-01 -1.38492378e-02 -1.07595003e-01
  3.94347822e-01  3.08482884e-01 -1.99013782e-01 -1.19791667e-01
  3.36509974e-01  1.23998754e-01  7.09777787e-02 -2.24189107e-01
  6.39129491e-02 -1.12729096e-01  3.77303329e-01  3.00875850e-01
 -2.07118920e-01 -9.94466146e-02  3.56407399e-01  1.32717900e-01
  6.50981103e-02 -2.35471351e-01  3.08298796e-02 -1.13459936e-01
  4.02763870e-01  1.86608654e-01 -1.28017562e-01 -5.94654983e-02
  2.04499931e-01 -1.47962672e-01  9.35256954e-02  4.85017271e-02
 -1.49293249e-01 -7.20514017e-02  4.75762559e-02  2.64251337e-02
 -7.770347

### 2) The structure wrangler
The `StructureWrangler` is a class that will is used to create and organize the data that will be used to train (and possibly test) the cluster expansion. It makes sure that all the supplied structures appropriately match the prim structure, and obtains the necessary information to correctly normalize target properties (such as energy) necessary for training.

Training data is added to a `StructureWrangler` using `ComputedStructureEntry` instances from `pymatgen`.

Matching relaxed structures can be a tricky problem, especially for ionic systems with vacancies. See the notebook on structure matching for tips on how to tweak parameters.

In [14]:
wrangler = StructureWrangler(subspace)

# the energy is taken directly from the ComputedStructureEntry
# any additional properties can also be added, see notebook on
# training data preparation for an example.
for entry in lno_entries:
    wrangler.add_entry(entry, verbose=True)
# The verbose flag will print structures that fail to match.

print(f'\nTotal structures that match {wrangler.num_structures}/{len(lno_entries)}')

/opt/homebrew/Caskroom/miniforge/base/envs/smol/lib/python3.10/site-packages/smol/cofe/wrangling/wrangler.py:808: UserWarning: The following structures have duplicated correlation vectors:
 Index 18 - Zr1 Ti5 V51 Cr5 W2 energy=-576.02983151
Index 222 - Zr1 Ti5 V51 Cr5 W2 energy=-576.02981629
 Consider adding more terms to the clustersubspace or filtering duplicates.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/smol/lib/python3.10/site-packages/smol/cofe/wrangling/wrangler.py:808: UserWarning: The following structures have duplicated correlation vectors:
 Index 16 - Zr1 Ti3 V53 Cr6 W1 energy=-575.52894581
Index 226 - Zr1 Ti3 V53 Cr6 W1 energy=-575.52882792
 Consider adding more terms to the clustersubspace or filtering duplicates.
  warnings.warn(



Total structures that match 230/230


## 3) Training

Training a cluster expansion is one of the most critical steps. This is how you get **effective cluster interactions (ECI's)**. To do so you need an estimator class that implements some form of regression model. In this case we will use simple least squares regression using the `LinearRegression` estimator from `scikit-learn`.

In `smol` the coefficients from the fit are not exactly the ECI's but the ECI times the multiplicity of their orbit.

In [15]:
from sklearn.linear_model import LinearRegression
# Set fit_intercept to False because we already do this using
# the empty cluster.
estimator = LinearRegression(fit_intercept=False)
estimator.fit(wrangler.feature_matrix, wrangler.get_property_vector('energy'))
coefs = estimator.coef_

#### 3.1) Check the quality of the fit
There are many ways to evaluate the quality of a fit. The simplest involve stadard training set prediction error metrics. But when evaluating a CE more seriously we need to consider further metrics and how the CE will be used.
Here we will just look at in sample mean squared error and max error.

In [16]:
from sklearn.metrics import mean_squared_error, max_error

train_predictions = np.dot(wrangler.feature_matrix, coefs)

rmse = mean_squared_error(
    wrangler.get_property_vector('energy'), train_predictions, squared=False
)
maxer = max_error(wrangler.get_property_vector('energy'), train_predictions)

print(f'RMSE {1E3 * rmse} meV/prim')
print(f'MAX {1E3 * maxer} meV/prim')

RMSE 1.609535181138332 meV/prim
MAX 6.674743955077389 meV/prim


/opt/homebrew/Caskroom/miniforge/base/envs/smol/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 4) The cluster expansion
Now we can use the above work to create the `ClusterExpansion`. The cluster expansion can be used to predict the fitted property for new structures, either for testing quality or for simulations such as in Monte Carlo.
Note that when using the `predict` function, the cluster expansion will have to match the given structure if it has not seen it before.
We will also store the details of the regression model used to fit the cluster expansion by using a `RegressionData` object.

In [17]:
reg_data = RegressionData.from_sklearn(
    estimator, wrangler.feature_matrix,
    wrangler.get_property_vector('energy')
)


expansion = ClusterExpansion(
    subspace, coefficients=coefs, regression_data=reg_data
)

structure = random.choice(wrangler.structures)
prediction = expansion.predict(structure, normalized=True)

print(
    f'The predicted energy for a structure with composition '
    f'{structure.composition} is {prediction} eV/prim.\n'
)
print(f'The fitted coefficients are:\n{expansion.coefs}\n')
print(f'The effective cluster interactions are:\n{expansion.eci}\n')
print(expansion)

The predicted energy for a structure with composition Zr1 Ti5 V54 Cr1 W3 is -18.04345443828815 eV/prim.

The fitted coefficients are:
[-3.94981606e+09  1.30548251e+10 -2.71731808e+09  7.91305558e+09
  6.43933527e+09 -7.66794122e+09  8.38069015e+09 -5.52851530e+09
 -1.11861651e+10 -1.81064565e+09  7.59340250e+09 -1.18556131e+09
  4.17452027e+09 -4.69353631e+09 -3.14182740e+09 -7.43761235e+09
  4.11079034e+09 -1.30654142e+10 -2.85841720e+09 -1.12281337e+09
 -5.56599763e+08 -2.20841370e+09 -7.97197328e+09 -2.81145514e+09
 -3.30878738e+09 -5.35471916e-01 -4.03120995e-01 -5.36379814e-01
 -4.24032211e-02 -4.74834442e-02 -3.38005066e-01 -1.04322433e-02
 -1.25691414e-01 -2.33441830e-01 -7.08398819e-02 -7.06146717e-01
 -7.31519222e-01 -1.60940409e-01  1.63133621e-01 -1.54805660e-01
 -3.90973568e-01  3.41699421e-02  2.96133518e-01 -4.07996178e-02
 -3.14350128e-02  3.59915584e+08 -2.23002470e+10  1.55815000e+10
  7.11632739e+09 -1.02239908e+10  9.20133327e+08 -1.14618415e+10
  5.42660853e+09 -1.3

### 5) Saving your work
All core classes in `smol` are `MSONables` and so can be saved using their `as_dict` methods or better yet with `monty.serialization.dumpfn`.

Currently there is also a convenience function in `smol` that will nicely save all of your work for you in a standardized way. Work saved with the `save_work` function is saved as a dictionary with standardized names for the classes. Since a work flow should only contain 1 of each core classes the function will complain if you give it two of the same class (i.e. two wranglers)

In [18]:
from smol.io import save_work
import os 

#file_path = 'v4cr4ti_fin_work.mson'
expansions_path = '/Users/myless/Packages/structure_maker/Expansions'
file_path = 'v1_6cr1_6ti1_6zr1_6w_fin_work.mson'
# we can save the subspace as well, but since both the wrangler
# and the expansion have it, there is no need to do so.
save_work(os.path.join(expansions_path,file_path), wrangler, expansion)

#### 5.1) Loading previously saved work

In [12]:
from smol.io import load_work

work = load_work(file_path)
for name, obj in work.items():
    print(f'{name}: {type(obj)}\n')

StructureWrangler: <class 'smol.cofe.wrangling.wrangler.StructureWrangler'>

ClusterExpansion: <class 'smol.cofe.expansion.ClusterExpansion'>

